## Take Action: Select portfolio for analysis

In [ ]:
portfolio_id = "041187df-881e-4a26-8d1e-0d0ad11e3938"
account_id = 98
initial_amount = 10000.0

### Imports

In [ ]:
import matplotlib.pyplot as plt
import empyrical as ep
import math
import pandas as pd
import numpy as np
from IPython.display import HTML, display, Markdown
from liualgotrader.analytics import analysis
import matplotlib.ticker as mtick
%matplotlib inline

### Load data details

In [ ]:
portfolio_returns = analysis.calc_portfolio_returns(account_id, initial_amount, portfolio_id)

In [ ]:
portfolio_returns

In [ ]:
spy_returns = analysis.compare_to_symbol_returns(portfolio_id, "SPY")

In [ ]:
spy_returns

### Data Prep

In [ ]:
def get_max_drawdown_underwater(underwater):
    """
    !!!!!!!!!!!!!!!!
    !!FROM PYFOLIO!!
    !!!!!!!!!!!!!!!!
    Determines peak, valley, and recovery dates given an 'underwater'
    DataFrame.
    An underwater DataFrame is a DataFrame that has precomputed
    rolling drawdown.
    Parameters
    ----------
    underwater : pd.Series
       Underwater returns (rolling drawdown) of a strategy.
    Returns
    -------
    peak : datetime
        The maximum drawdown's peak.
    valley : datetime
        The maximum drawdown's valley.
    recovery : datetime
        The maximum drawdown's recovery.
    """

    valley = underwater.idxmin()  # end of the period
    # Find first 0
    peak = underwater[:valley][underwater[:valley] == 0].index[-1]
    # Find last 0
    try:
        recovery = underwater[valley:][underwater[valley:] == 0].index[0]
    except IndexError:
        recovery = np.nan  # drawdown not recovered
    return peak, valley, recovery


def get_top_drawdowns(returns, top=10):
    """

    Finds top drawdowns, sorted by drawdown amount.
    Parameters
    ----------
    returns : pd.Series
        Daily returns of the strategy, noncumulative.
         - See full explanation in tears.create_full_tear_sheet.
    top : int, optional
        The amount of top drawdowns to find (default 10).
    Returns
    -------
    drawdowns : list
        List of drawdown peaks, valleys, and recoveries. See get_max_drawdown.
    """

    returns = returns.copy()
    df_cum = ep.cum_returns(returns, 1.0)
    running_max = np.maximum.accumulate(df_cum)
    underwater = df_cum / running_max - 1

    drawdowns = []
    for _ in range(top):
        peak, valley, recovery = get_max_drawdown_underwater(underwater)
        # Slice out draw-down period
        if not pd.isnull(recovery):
            underwater.drop(underwater[peak: recovery].index[1:-1],
                            inplace=True)
        else:
            # drawdown has not ended yet
            underwater = underwater.loc[:peak]

        drawdowns.append((peak, valley, recovery))
        if ((len(returns) == 0)
                or (len(underwater) == 0)
                or (np.min(underwater) == 0)):
            break

    return drawdowns

def my_rolling_sharpe(y):
    return np.sqrt(126) * (y.mean() / y.std()) # 21 days per month X 6 months = 126

In [ ]:
me_pct = portfolio_returns.totals.pct_change(1)
me = me_pct.add(1).cumprod().dropna().sub(1).mul(100)
portfolio_returns["volatility"] = portfolio_returns.totals.pct_change(1).rolling(20).std()
my_volatility = portfolio_returns["volatility"].cumsum().dropna()

In [ ]:
portfolio_returns.totals.pct_change(1).dropna().add(1).cumprod().dropna().sub(1).mul(100)

In [ ]:
market = pd.DataFrame(spy_returns.pct_change(1).dropna().add(1).cumprod().dropna().sub(1).mul(100))
market["volatility"] = spy_returns.pct_change(1).rolling(20).std()
market_volatility = market["volatility"].cumsum().dropna()

In [ ]:
drawdown_periods = get_top_drawdowns(me_pct, top=5)

In [ ]:
portfolio_returns['rs'] = portfolio_returns.totals.pct_change(1).rolling(60).apply(my_rolling_sharpe).dropna()

## Visuals

### Portfolio Summary

In [ ]:
fig= plt.figure(figsize=(20,15))
ax1 = plt.subplot2grid(shape=(4, 1), loc=(0, 0), rowspan=2)
ax2 = plt.subplot2grid(shape=(4, 1), loc=(2, 0), rowspan=1)
#ax3 = plt.subplot2grid(shape=(4, 1), loc=(3, 0), rowspan=1)
fig.subplots_adjust(hspace=0.5)
ax1.plot(portfolio_returns.totals, color="green", linewidth=3.0, label='total')
ax1.plot(portfolio_returns['equity'], color="green", linewidth=1.0, label='equity')
ax1.set_title("Portfolio $ Value over time", fontsize=16)
ax2.plot(me_pct, color="black", linewidth=3.0)
for box in drawdown_periods:
    x2 = str(box[1])
    ax2.fill_between(x=[str(box[0]), x2 if x2 != 'nan' else str(me_pct.index[-1])], y1=me_pct.max(), y2=me_pct.min(), alpha=0.5)
#ax2.set_title("Drawdown periods", fontsize=16)
#ax3.set_title("Rolling (60 Days) Sharpe Ratio", fontsize=14)
#ax3.plot(portfolio_returns['rs'], color="black", linewidth=3.0)
plt.show()

### Benchmark

In [ ]:
fig= plt.figure(figsize=(20,12))
ax1 = plt.subplot2grid(shape=(3, 1), loc=(0, 0), rowspan=2)
ax2 = plt.subplot2grid(shape=(3, 1), loc=(2, 0), rowspan=1)
fig.subplots_adjust(hspace=0.5)
ax1.plot(me, label ="me", color="green", linewidth=3.0)
ax1.plot(market.SPY, label ="SP500", color="black")
ax1.set_title("My Algo vs. SP500 index", fontsize=16)
plt.xticks(rotation=45)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax1.grid(color='gray', linestyle='-', linewidth=0.5)
ax1.legend()
ax2.plot(portfolio_returns["volatility"].dropna(), label="me", color="green", linewidth=3.0) 
ax2.plot(market["volatility"].dropna(), label="market", color="black") 
ax2.set_title("Volatility", fontsize=16)
ax2.legend
plt.show()